# Simulation and Visualization of Path Integral

path-integral derivation of one-dimentional diffusion process
$$
\begin{align}
P(x_N, t_N) &= \frac{1}{2\pi} \int^{\infty}_{-\infty} \rm{d}x_0 \sqrt{\frac{\pi}{D(t_N - t_0)}} 
\exp \left\{ - \frac{[\ (x_N - x_0) - u(t_N - t_0) \ ]^2}{4D(t_N-t_0)} \right\} \delta(x_0-x_I) \nonumber \\
            &= \sqrt{\frac{1}{4\pi D(t_N - t_0)}} \exp \left\{ - \frac{[\ (x_N - x_I) - u(t_N - t_0) \ ]^2}{4D(t_N-t_0)} \right\} \nonumber
\end{align}
$$

$\mu$ : $$ x_I + u(t_N - t_0) $$

$\sigma$ : $$ \sqrt{2 D (t_N - t_0) } $$

In [ ]:
# basic
import numpy as np
import matplotlib.pyplot as plt

# animation
from matplotlib import patches
from matplotlib.animation import ArtistAnimation
from IPython import display

In [ ]:
def P(
      xn_xi: np.ndarray,
      tn_t0: float, 
      D: float,
      u: float,
    ):
    """
    xn_xi : x_N - x_I [m]
    tn_t0 : t_N - t_0 [sec]
    D : diffusion coefficient
    u : velocity
    """

    p = np.sqrt(1.0 / (4.0 * np.pi * D * tn_t0) ) *\
        np.exp(-((xn_xi) - u * (tn_t0))**2 / (4.0 * D * tn_t0))

    return p

In [ ]:
# simulation params
num_of_samples = 401
xmin, xmax = -8.0, +8.0 # [m]

xi = 1.0 # [m]
x = np.linspace(xmin, xmax, num_of_samples)
x_xi = x - np.ones(num_of_samples) * xi
t0 = 0.0 # [sec]
t = 0.1 # [sec]
t_t0 = t - t0
D = 1.0 # [m^2/s]
u = 5.0 # [m/s]

# make figure
fig, ax = plt.subplots()

# layout settings
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# set axis limits
ax.set_xlim(xmin, xmax)
ax.set_ylim( 0,  1.0)

# draw x = x_i
ax.axvline(x=xi, linestyle="dashed", color="gray")

mean = xi + u*(t_t0)
sigma = np.sqrt(2*D*t_t0)

ax.axvline(x=-10 * sigma + mean, linestyle="dashed", color="gray")
ax.axvline(x=+10 * sigma + mean, linestyle="dashed", color="gray")

# draw P(x) with fill & plot
ax.fill_between(x, P(x_xi, t_t0, D, u), fc='#add8e6')
ax.plot(x, P(x_xi, t_t0, D, u), color="#005AFF")

# show plot
plt.show()

In [ ]:
# prepare figure
fig, ax = plt.subplots(1, 1, figsize=(6,3))

# simulation params
t0 = 0.0 # [sec]
t = 10.0 # [sec]
D = 1.0 # [m^2/s]
u = 10.0 # [m/s]
xi = 1.0 # [m]
dt = 0.05 # [s]
dx = 0.05 # [m]

# simulation settings
num_sigma = 3 # x range to consider in simulation
t_t0 = t - t0
sim_steps = int(t_t0 // dt) # [step]
xlim0, xlimn = xi-np.sign(u)*np.sqrt(2*D*t)*num_sigma, xi+u*t+np.sign(u)*np.sqrt(2*D*t)*num_sigma
xmin, xmax = min(xlim0, xlimn), max(xlim0, xlimn)
num_of_samples = int((xmax-xmin)/dx)
x = np.linspace(xmin, xmax, num_of_samples)
x_xi = x - np.ones(num_of_samples) * xi

# graph layout settings
ax.set_xlabel("x [m]")
ax.set_ylabel("Probability P(x)")
ax.set_xlim(xmin, xmax)
ymax = np.max(P(x_xi, dt, D, u))*1.5
ax.set_ylim(0, ymax)
ax.set_yticks([0, 1])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.tight_layout()

# simulation loop
frames = []
for i in range(1, sim_steps):
    current_t = t0 + i * dt
    frame = ax.plot(x, P(x_xi, current_t, D, u), color="#005AFF")
    frame += [ax.fill_between(x, P(x_xi, current_t, D, u), fc='#add8e6')]
    text = "Time t = {t:>4.1f} [s]".format(t=current_t)
    frame += [ax.text(0.5, 0.9, text, ha='center', transform=ax.transAxes, fontsize=10, fontfamily='monospace')]
    frames.append(frame)

# show animation
ani = ArtistAnimation(fig, frames, interval=dt*1000)
html = display.HTML(ani.to_jshtml())
display.display(html)
plt.close()

# save animation
filename = f"diffusion_xi{xi}_t{t_t0}_D{D}_u{u}.mp4"
ani.save(filename, writer="ffmpeg")